In [5]:
import os
import json
import glob
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from PIL import Image, ImageOps
from skimage import io
from skimage.color import rgba2rgb, rgb2xyz
from tqdm import tqdm
from dataclasses import dataclass
from math import floor, ceil
import random

# Train data generation
import collections
import csv
from pathlib import Path
from typing import List, Tuple, Any

import time
import re
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import lightgbm as lgb
import pickle

import math
import scipy.stats as stats

import multiprocessing
from multiprocessing import Pool

# from google.cloud import storage

pd.set_option("display.max_columns", 100)

In [3]:
train_files = sorted(glob.glob("./train_2/*"))
test_files = sorted(glob.glob("./test_2/*"))
# print(train_files[0], train_files[8], train_files[16], train_files[23])
# print(test_files[0], test_files[8], test_files[16], test_files[23])

In [4]:
!ls ./train_2

5a0546857ecc773753327266_train.csv  5da138274db8ce0c98bbd3d2_train.csv
5c3c44b80379370013e0fd2b_train.csv  5da1382d4db8ce0c98bbe92e_train.csv
5d27075f03f801723c2e360f_train.csv  5da138314db8ce0c98bbf3a0_train.csv
5d27096c03f801723c31e5e0_train.csv  5da138364db8ce0c98bc00f1_train.csv
5d27097f03f801723c320d97_train.csv  5da1383b4db8ce0c98bc11ab_train.csv
5d27099f03f801723c32511d_train.csv  5da138754db8ce0c98bca82f_train.csv
5d2709a003f801723c3251bf_train.csv  5da138764db8ce0c98bcaa46_train.csv
5d2709b303f801723c327472_train.csv  5da1389e4db8ce0c98bd0547_train.csv
5d2709bb03f801723c32852c_train.csv  5da138b74db8ce0c98bd4774_train.csv
5d2709c303f801723c3299ee_train.csv  5da958dd46f8266d0737457b_train.csv
5d2709d403f801723c32bd39_train.csv  5dbc1d84c1eb61796cf7c010_train.csv
5d2709e003f801723c32d896_train.csv  5dc8cea7659e181adb076a3f_train.csv


In [11]:
# 4 paths x 20 cols -> 42.8 sec w/ colab (cpu_count=2)
# 4 paths x 20 cols -> 50.6 sec w/ gcp (cpu_count=16)
# 16 paths x 20 cols -> 48.4 sec w/ gcp (cpu_count=16)
# 24 paths x 20 cols -> 50.9 sec w/ gcp (cpu_count=16)
# 24 paths x 20 cols -> 51.5 sec w/ gcp w/ path_split (cpu_count=16)
# 24 paths x all cols -> 3476.9 sec

def check_nan_ratio(path):
    res = []
    df = pd.read_csv(path)
    path_split = path.split("/")
    path_colab = "./colab_indoor/train_2/" + path_split[-1]
    print(path_colab)
    df = df.iloc[:, 11:] # only the wifi bssid columns
    cols = list(df.columns)
    # cols = random.sample(cols, 20) # comment out when running fully
    for col in cols:
        perc = len(df[df[col] == -999]) / len(df)
        if perc > 0.98:
            res.append([path_colab, col, perc])
    return res

start = time.time()

num_cores = multiprocessing.cpu_count()
print(f"num_cores={num_cores}")
pool = Pool(num_cores)

# train_samples = random.sample(train_files, 16)
# res_all = pool.map(check_nan_ratio, train_samples)
res_all = pool.map(check_nan_ratio, train_files)

res_unpack = [row for res in res_all for row in res]
nan_ratio_df = pd.DataFrame(res_unpack, columns=["path", "col", "perc"])
nan_ratio_df.to_csv('./drop_col_lookup.csv', index=False)
display(nan_ratio_df.head())

print("time to process: ", time.time() - start)

num_cores=16
./colab_indoor/train_2/5da138274db8ce0c98bbd3d2_train.csv
./colab_indoor/train_2/5da138364db8ce0c98bc00f1_train.csv
./colab_indoor/train_2/5d27099f03f801723c32511d_train.csv
./colab_indoor/train_2/5d2709a003f801723c3251bf_train.csv
./colab_indoor/train_2/5da138314db8ce0c98bbf3a0_train.csv
./colab_indoor/train_2/5d2709e003f801723c32d896_train.csv
./colab_indoor/train_2/5d2709d403f801723c32bd39_train.csv
./colab_indoor/train_2/5d2709b303f801723c327472_train.csv
./colab_indoor/train_2/5da1382d4db8ce0c98bbe92e_train.csv
./colab_indoor/train_2/5da1383b4db8ce0c98bc11ab_train.csv
./colab_indoor/train_2/5d27097f03f801723c320d97_train.csv
./colab_indoor/train_2/5da138754db8ce0c98bca82f_train.csv
./colab_indoor/train_2/5c3c44b80379370013e0fd2b_train.csv
./colab_indoor/train_2/5a0546857ecc773753327266_train.csv
./colab_indoor/train_2/5d2709bb03f801723c32852c_train.csv
./colab_indoor/train_2/5da138764db8ce0c98bcaa46_train.csv
./colab_indoor/train_2/5d27096c03f801723c31e5e0_train.csv
.

,path,col,perc
0,./colab_indoor/train_2/5a0546857ecc77375332726...,4608951935961c98acdd8a88cb4d3dfd0358fa6c,0.981713
1,./colab_indoor/train_2/5a0546857ecc77375332726...,5eb37c76fee8a970b031fe2912eb15e77797d6a5,0.982035
2,./colab_indoor/train_2/5a0546857ecc77375332726...,acccd377c64a35e52d69b9e13f2f28f80a7d436e,0.981820
3,./colab_indoor/train_2/5a0546857ecc77375332726...,c3b4a3b6f118ac96d0e8c2321a4a20b73108152d,0.985478
4,./colab_indoor/train_2/5a0546857ecc77375332726...,ed719a231a8a82b0f288525a1c08fe3178ab2aeb,0.998064


time to process:  3476.9365234375
